# Mouse Facial Expressions - 학습 노트북 (수정됨)

이 노트북은 처음부터 다시 시작하여 모든 경로를 확인하고 학습을 진행합니다.

## ✅ 수정 사항
- ModelCheckpoint 에러 수정: `val_acc` 메트릭을 Lightning에 올바르게 로깅하도록 수정했습니다.

## Step 1: 현재 상태 확인

In [1]:
import os
import sys

# 프로젝트 디렉토리로 이동
os.chdir('/Users/minakang/Desktop/mouse-facial-expressions-2023-main')
print(f"현재 디렉토리: {os.getcwd()}")
print(f"Python 버전: {sys.version}")

현재 디렉토리: /Users/minakang/Desktop/mouse-facial-expressions-2023-main
Python 버전: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 11:23:37) [Clang 14.0.6 ]


In [2]:
# 필수 라이브러리 확인
import torch
import lightning
import mlflow

print(f"PyTorch 버전: {torch.__version__}")
print(f"Lightning 버전: {lightning.__version__}")
print(f"MLflow 버전: {mlflow.__version__}")
print(f"\nCUDA 사용 가능: {torch.cuda.is_available()}")
print(f"MPS 사용 가능: {torch.backends.mps.is_available()}")
print(f"\n⚠️ 이 노트북은 CPU로 학습합니다 (안정성을 위해)")

PyTorch 버전: 2.9.1
Lightning 버전: 2.5.6
MLflow 버전: 3.6.0

CUDA 사용 가능: False
MPS 사용 가능: True

⚠️ 이 노트북은 CPU로 학습합니다 (안정성을 위해)


In [3]:
# 데이터 폴더 확인
!ls -la data/

total 0
drwx------    8 minakang  staff   256 Nov 23 16:01 .
drwx------@  48 minakang  staff  1536 Nov 24 08:45 ..
drwx------    3 minakang  staff    96 Nov 23 15:30 dlc
drwx------@ 290 minakang  staff  9280 Nov 22 16:28 frames_backup
drwx------    3 minakang  staff    96 Nov 23 15:30 interim
drwx------  289 minakang  staff  9248 Nov 23 15:30 local_frames
drwx------    4 minakang  staff   128 Nov 23 15:30 processed
drwx------    5 minakang  staff   160 Nov 23 15:31 raw


## Step 2: 데이터 확인

현재 사용 가능한 프레임 데이터를 확인합니다.

In [4]:
# 사용 가능한 프레임 폴더 확인
import os

frames_folders = []
if os.path.exists('data/frames_backup'):
    frames_folders.append('data/frames_backup')
if os.path.exists('data/local_frames'):
    frames_folders.append('data/local_frames')

if frames_folders:
    print("✅ 사용 가능한 프레임 폴더:")
    for folder in frames_folders:
        frame_count = sum(1 for root, dirs, files in os.walk(folder) for f in files if f.endswith('.png'))
        print(f"  - {folder}: {frame_count:,} 프레임")
else:
    print("❌ 프레임 폴더를 찾을 수 없습니다.")
    print("\n다음 중 하나를 수행하세요:")
    print("1. Google Drive에서 데이터를 복사하거나")
    print("2. data/local_frames 폴더에 프레임 데이터가 있는지 확인하세요")

✅ 사용 가능한 프레임 폴더:
  - data/frames_backup: 57,174 프레임
  - data/local_frames: 57,174 프레임


## Step 3: .env 파일 설정

환경 변수를 설정합니다. 프레임 폴더 경로를 자동으로 감지합니다.

In [5]:
# 프레임 폴더 자동 감지
import os

project_dir = '/Users/minakang/Desktop/mouse-facial-expressions-2023-main'

# 우선순위: frames_backup > local_frames
if os.path.exists(f'{project_dir}/data/frames_backup'):
    frames_folder = f'{project_dir}/data/frames_backup'
    print("✅ frames_backup 폴더를 사용합니다")
elif os.path.exists(f'{project_dir}/data/local_frames'):
    frames_folder = f'{project_dir}/data/local_frames'
    print("✅ local_frames 폴더를 사용합니다")
else:
    frames_folder = f'{project_dir}/data/local_frames'
    print("⚠️ 프레임 폴더를 찾을 수 없습니다. local_frames를 기본값으로 설정합니다.")

# .env 파일 생성
env_content = f"""MFE_RAW_VIDEO_FOLDER={project_dir}/data/raw
MFE_RAW_CSV_FOLDER={project_dir}/data/raw
MFE_VERSION=20230627
MFE_PROCESSED_VIDEO_FOLDER={project_dir}/data/processed
MFE_EXTRACTED_FRAMES_FOLDER={frames_folder}
MFE_TASKS={project_dir}/data/processed
"""

with open('.env', 'w') as f:
    f.write(env_content)

print("\n✅ .env 파일 생성 완료!")
print("\n내용:")
print(env_content)

✅ frames_backup 폴더를 사용합니다

✅ .env 파일 생성 완료!

내용:
MFE_RAW_VIDEO_FOLDER=/Users/minakang/Desktop/mouse-facial-expressions-2023-main/data/raw
MFE_RAW_CSV_FOLDER=/Users/minakang/Desktop/mouse-facial-expressions-2023-main/data/raw
MFE_VERSION=20230627
MFE_PROCESSED_VIDEO_FOLDER=/Users/minakang/Desktop/mouse-facial-expressions-2023-main/data/processed
MFE_EXTRACTED_FRAMES_FOLDER=/Users/minakang/Desktop/mouse-facial-expressions-2023-main/data/frames_backup
MFE_TASKS=/Users/minakang/Desktop/mouse-facial-expressions-2023-main/data/processed



## Step 4: 데이터셋 생성 (필요한 경우만 실행)

⚠️ 이미 `data/processed/task-1.1/` 디렉토리에 `.pkl` 파일들이 있다면 이 단계를 건너뛰세요.

In [6]:
# 데이터셋 파일 확인
import os

processed_dir = 'data/processed/task-1.1'
if os.path.exists(processed_dir):
    pkl_files = [f for f in os.listdir(processed_dir) if f.endswith('.pkl')]
    if len(pkl_files) > 0:
        print(f"✅ 데이터셋이 이미 존재합니다 ({len(pkl_files)}개 파일)")
        print("\n파일 목록:")
        !ls -lh data/processed/task-1.1/
    else:
        print("❌ 데이터셋 파일이 없습니다. 아래 셀을 실행하여 생성하세요.")
else:
    print("❌ 데이터셋 디렉토리가 없습니다. 아래 셀을 실행하여 생성하세요.")

✅ 데이터셋이 이미 존재합니다 (10개 파일)

파일 목록:
total 12040
-rw-------  1 minakang  staff   1.5M Nov 23 16:01 dataset_df.pkl
-rw-------  1 minakang  staff   168K Nov 23 16:02 eval.pkl
-rw-------  1 minakang  staff   538K Nov 23 16:01 fold0.pkl
-rw-------  1 minakang  staff   538K Nov 23 16:01 fold1.pkl
-rw-------  1 minakang  staff   538K Nov 23 16:01 fold2.pkl
-rw-------  1 minakang  staff   538K Nov 23 16:01 fold3.pkl
-rw-------  1 minakang  staff   538K Nov 23 16:01 fold4.pkl
-rw-------  1 minakang  staff   538K Nov 23 16:02 fold5.pkl
-rw-------  1 minakang  staff   538K Nov 23 16:02 fold6.pkl
-rw-------  1 minakang  staff   538K Nov 23 16:02 fold7.pkl
-rw-------@ 1 minakang  staff   395B Nov 23 16:01 README.txt


In [7]:
# 데이터셋 생성 (약 5-10분 소요)
# 위 셀에서 데이터셋이 없다고 표시된 경우에만 실행하세요
!python mouse_facial_expressions/data/make_datasets.py task1 --version "1.1" --frameset_size 1

2025-11-24 08:47:10,730 - __main__ - INFO - 
Task1 classification task using sets of 1 frames

Classes:
- 0: All animals at preinjection
- 1: LPS High dose at 4 hours

To balance the training datset (n=10000), classes are first sampled
randomly then videos are randomly sampled and finally frames.

The testing dataset (n=1000) only samples by video, preserving imbalances
in the dataset.

Split over 8 stratified kfolds grouped by mouse.

2025-11-24 08:47:10,730 - __main__ - INFO - Seeding 13641
2025-11-24 08:47:10,730 - __main__ - INFO - Loading treatment csv
Listing directories in /Users/minakang/Desktop/mouse-facial-expressions-2023-main/data/frames_backup...
Found 287 directories. Listing files in parallel...
Listing files: 100%|████████████████████████| 287/287 [00:00<00:00, 1095.45it/s]
Found 57174 files.
2025-11-24 08:47:11,583 - __main__ - INFO - Removing control mouse (m18) which was identified as having pain/sickness symptoms before experiment start
2025-11-24 08:47:11,592 - __m

## Step 5: 학습 시작!

### Option A: 1 epoch 테스트 (~40-50분)

먼저 1 epoch로 테스트하여 모든 것이 정상적으로 작동하는지 확인하는 것을 권장합니다.

In [ ]:
# 1 epoch 테스트
!python mouse_facial_expressions/models/train_task1_baseline_model.py --epochs 1 --dataset_version "1.1"

2025-11-24 08:47:46,927 - __main__ - INFO - beginning model run train_task1_baseline_model.py
/opt/anaconda3/lib/python3.13/site-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/11/24 08:47:47 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Cmd('git') failed due to: exit code(1)
  cmdline: git version
  stderr: 'xcode-select: note: No developer tools were found, requesting install.
If developer tools are located at a non-default location on disk, use `sudo xcode-select --switch path/to/Xcode.app` to specify the Xcode that you wish to use for command line developer tools, and cancel the installation dialog.
See `man 

### Option B: 전체 학습 10 epochs (~7-8시간)

⚠️ 이 셀은 오래 걸립니다. 밤새 실행하는 것을 권장합니다.

In [ ]:
# 10 epochs 전체 학습
!python mouse_facial_expressions/models/train_task1_baseline_model.py --epochs 10 --dataset_version "1.1"

## Step 6: 학습 모니터링

학습이 진행되는 동안 다른 셀에서 진행 상황을 확인할 수 있습니다.

In [ ]:
# 체크포인트 확인
import os
import glob

checkpoint_dir = 'models/checkpoints'
if os.path.exists(checkpoint_dir):
    checkpoints = glob.glob(f'{checkpoint_dir}/*.ckpt')
    if checkpoints:
        print(f"✅ 저장된 체크포인트 ({len(checkpoints)}개):")
        !ls -lht models/checkpoints/
    else:
        print("아직 체크포인트가 없습니다.")
else:
    print("체크포인트 디렉토리가 아직 생성되지 않았습니다.")

In [ ]:
# MLflow 실험 결과 확인
import mlflow
import os

mlflow.set_tracking_uri("file://" + os.path.join(os.getcwd(), "mlruns"))

client = mlflow.tracking.MlflowClient()
experiments = client.search_experiments()

if experiments:
    print("MLflow 실험:")
    for exp in experiments:
        print(f"\n실험 ID: {exp.experiment_id}")
        print(f"이름: {exp.name}")
        
        runs = client.search_runs(exp.experiment_id, max_results=5)
        if runs:
            print(f"\n최근 실행 ({len(runs)}개):")
            for run in runs:
                print(f"  - Run ID: {run.info.run_id[:8]}...")
                print(f"    상태: {run.info.status}")
                if run.data.metrics:
                    print(f"    메트릭: {dict(list(run.data.metrics.items())[:3])}")
else:
    print("아직 MLflow 실험이 없습니다.")

## Step 7: 결과 확인

In [ ]:
# 최고 성능 모델 찾기
import glob
import re
import os

checkpoints = glob.glob('models/checkpoints/*.ckpt')
if checkpoints:
    best_acc = 0
    best_ckpt = None
    
    for ckpt in checkpoints:
        match = re.search(r'val_acc=([0-9.]+)', ckpt)
        if match:
            acc = float(match.group(1))
            if acc > best_acc:
                best_acc = acc
                best_ckpt = ckpt
    
    if best_ckpt:
        print(f"🏆 최고 성능 모델:")
        print(f"   파일: {os.path.basename(best_ckpt)}")
        print(f"   Validation Accuracy: {best_acc:.2%}")
        
        # 파일 크기 확인
        size_mb = os.path.getsize(best_ckpt) / (1024 * 1024)
        print(f"   크기: {size_mb:.1f} MB")
    else:
        print("체크포인트 파일명에서 val_acc를 찾을 수 없습니다.")
        print("\n저장된 체크포인트:")
        for ckpt in checkpoints:
            print(f"  - {os.path.basename(ckpt)}")
else:
    print("아직 체크포인트가 없습니다.")

## 유용한 팁

### 학습 중단하기
- Jupyter 메뉴: Kernel → Interrupt
- 또는 셀 실행 중 ■ 버튼 클릭

### 학습 재개하기
체크포인트에서 학습을 재개하려면 (아직 구현되지 않음):
```python
# 향후 추가 예정
```

### MLflow UI 실행
터미널에서:
```bash
cd /Users/minakang/Desktop/mouse-facial-expressions-2023-main
mlflow ui
```
그런 다음 브라우저에서 http://localhost:5000 접속

### 문제 해결

**에러: FileNotFoundError (프레임을 찾을 수 없음)**
- Step 3에서 .env 파일의 경로가 올바른지 확인
- `data/frames_backup` 또는 `data/local_frames` 폴더에 프레임이 있는지 확인

**에러: ModelCheckpoint could not find val_acc**
- ✅ 이 문제는 수정되었습니다! 학습 코드가 업데이트되었습니다.

**학습이 너무 느림**
- CPU로 학습하기 때문에 정상입니다
- 1 epoch당 약 40-50분 소요
- 밤새 실행하는 것을 권장합니다